In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
import sys
sys.path.insert(0,'..')
from utils import lossprettifier
from Classifier.VGG import *

Using TensorFlow backend.
C:\Users\Aleksander Podsiad\.conda\envs\WB_env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Aleksander Podsiad\.conda\envs\WB_env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Aleksander Podsiad\.conda\envs\WB_env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Aleks

In [3]:
x_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')
x_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

In [4]:
print(x_train.shape)
print(x_test.shape)

(300, 224, 224, 3)
(30, 224, 224, 3)


In [5]:
print(y_train.shape)
print(y_test.shape)

(300,)
(30,)


In [ ]:
#x_train /= 255????

In [8]:
from __future__ import print_function

import numpy as np
from random import shuffle
import time
import csv
from PIL import Image
import os
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras import initializers
from keras.optimizers import SGD
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import *
from keras.callbacks import CSVLogger
from keras import callbacks
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import sklearn.metrics as sklm
#import lossprettifier
#from VGG import *

In [9]:
# for reproducibility
np.random.seed(3768)

# use this environment flag to change which GPU to use 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # specify which GPU(s) to be used

#Get TensorFlow session
def get_session(): 
  config = tf.ConfigProto() 
  config.gpu_options.allow_growth = True 
  return tf.Session(config=config) 
  
# One hot encoding of labels 
def dense_to_one_hot(labels_dense,num_clases=4):
  return np.eye(num_clases)[labels_dense]

In [10]:
# Preparing training and test sets
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [11]:
y_train = dense_to_one_hot(y_train,num_clases=4)
y_valid= dense_to_one_hot(y_valid,num_clases=4)

In [12]:
#Image data generation for the training 
datagen = ImageDataGenerator(
               featurewise_center = False, 
               samplewise_center = False,  # set each sample mean to 0
               featurewise_std_normalization = True,  
               samplewise_std_normalization = False)  

datagen.fit(x_train) 
for i in range(len(x_test)):
      x_test[i] = datagen.standardize(x_test[i])

C:\Users\Aleksander Podsiad\.conda\envs\WB_env\lib\site-packages\keras_preprocessing\image\image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [13]:
#Defining hyperparameters
batch_Size = 32
steps_Per_Epoch = 32
numEpochs = 50

#Instantating VGG19 model
model = VGG19((224,224,3),4) #VGG19_dense for revised VGG19, VGG19 for VGG19. Please pay attention to VGG16(), chnage the input shape and class number in VGG.py.

#Creating an optimizers
adaDelta = keras.optimizers.Adadelta(lr=1.0, rho=0.95)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.95, nesterov=True)
model.compile(optimizer = sgd , loss = 'categorical_crossentropy', metrics = ['accuracy'])

#Creating early stopping 
earlystop = EarlyStopping(monitor = 'val_accuracy', min_delta = 0, patience = 50, verbose = 1, mode = 'auto', restore_best_weights = True)       

train_generator = datagen.flow(x_train, y_train, batch_size = batch_Size)
validation_generator = datagen.flow(x_valid, y_valid, batch_size = batch_Size)

# Model training
history = model.fit_generator(
    train_generator,
    steps_per_epoch = steps_Per_Epoch,
    validation_data = validation_generator, 
    validation_steps = 16,
    epochs = numEpochs,
    shuffle = True, 
    verbose = 1)



Epoch 1/50
32/32 [==============================] - 14s 435ms/step - loss: 1.1676 - accuracy: 0.3487 - val_loss: 1.1039 - val_accuracy: 0.2000
Epoch 2/50
32/32 [==============================] - 3s 89ms/step - loss: 0.9785 - accuracy: 0.5299 - val_loss: 1.0274 - val_accuracy: 0.6667
Epoch 3/50
32/32 [==============================] - 3s 92ms/step - loss: 0.7889 - accuracy: 0.6629 - val_loss: 0.9961 - val_accuracy: 0.6000
Epoch 4/50
32/32 [==============================] - 3s 92ms/step - loss: 0.6167 - accuracy: 0.7363 - val_loss: 1.2100 - val_accuracy: 0.6667
Epoch 5/50
32/32 [==============================] - 3s 87ms/step - loss: 0.6039 - accuracy: 0.7458 - val_loss: 1.4537 - val_accuracy: 0.5667
Epoch 6/50
32/32 [==============================] - 3s 91ms/step - loss: 0.4441 - accuracy: 0.8235 - val_loss: 1.5641 - val_accuracy: 0.5667
Epoch 7/50
32/32 [==============================] - 3s 92ms/step - loss: 0.3221 - accuracy: 0.8649 - val_loss: 1.7538 - val_accuracy: 0.6333
Epoch 8/5

In [14]:
modelPath = "VGG19_COVID19.h5"
resultPath = 'VGG19_COVID19.txt'

In [15]:
y_test_oh = dense_to_one_hot(y_test, num_clases=4)

# visualizing losses and accuracy
train_loss = history.history['loss']
val_loss = history.history['val_loss']

#Observing the losses but can be commented out as it's not mandatory 
reporter = lossprettifier.LossPrettifier(show_percentage=True)

for i in range(numEpochs-1):
    reporter(epoch=i, LossA = train_loss[i], LossAB = val_loss[i])

# Model evaluation 
score, acc = model.evaluate(x_test, y_test_oh, batch_size=batch_Size)
print("Accuracy:", acc)

#if acc>0.675:
model.save_weights(modelPath)

Epoch     0 | LossA: 1.17(+0.00%) 	| LossAB: 1.10(+0.00%) 	
Epoch     1 | LossA: 0.97(-16.69%) ▼	| LossAB: 1.03(-6.93%) ▼	
Epoch     2 | LossA: 0.79(-18.63%) ▼	| LossAB: 1.00(-3.05%) ▼	
Epoch     3 | LossA: 0.63(-20.86%) ▼	| LossAB: 1.21(+21.48%) ▲	
Epoch     4 | LossA: 0.61(-3.23%) ▼	| LossAB: 1.45(+20.13%) ▲	
Epoch     5 | LossA: 0.45(-25.83%) ▼	| LossAB: 1.56(+7.60%) ▲	
Epoch     6 | LossA: 0.33(-27.38%) ▼	| LossAB: 1.75(+12.13%) ▲	
Epoch     7 | LossA: 0.26(-21.01%) ▼	| LossAB: 1.60(-8.76%) ▼	
Epoch     8 | LossA: 0.38(+48.75%) ▲	| LossAB: 1.70(+6.22%) ▲	
Epoch     9 | LossA: 0.29(-24.04%) ▼	| LossAB: 3.94(+131.98%) ▲	
Epoch    10 | LossA: 0.36(+22.88%) ▲	| LossAB: 1.90(-51.87%) ▼	
Epoch    11 | LossA: 0.29(-19.09%) ▼	| LossAB: 2.09(+10.24%) ▲	
Epoch    12 | LossA: 0.32(+9.84%) ▲	| LossAB: 2.92(+39.56%) ▲	
Epoch    13 | LossA: 0.30(-6.90%) ▼	| LossAB: 1.71(-41.37%) ▼	
Epoch    14 | LossA: 0.37(+26.30%) ▲	| LossAB: 2.56(+49.34%) ▲	
Epoch    15 | LossA: 0.23(-37.15%) ▼	| LossAB: 2.44

In [16]:
y_pred = model.predict(x_test)
y_pred = y_pred.reshape(len(y_test), 4)
y_pred = np.argmax(y_pred, axis=1)

# Writing results on file
f = open(resultPath,'a') #create classification report
f.write(classification_report(y_test, y_pred))
f.write(str(sklm.cohen_kappa_score(y_test, y_pred))+","+str(acc)+","+str(score)+"\n")

#Print class-wise classification metrics
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.90      0.82        10
           1       0.60      0.90      0.72        10
           2       1.00      0.30      0.46        10

    accuracy                           0.70        30
   macro avg       0.78      0.70      0.67        30
weighted avg       0.78      0.70      0.67        30



In [17]:
f.close()